In [1]:
import openai
import json

client = openai.OpenAI()
messages = []

In [2]:
def get_weather(city):
    return f"The weather in {city} is sunny."


FUNCTION_MAP = {
    "get_weather": get_weather,
}


In [3]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the weather in a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The name of the city to get the weather for",
                    }
                },
                "required": ["city"],
            },
        },
    }
]

In [4]:
from openai.types.chat import ChatCompletionMessage


def process_ai_response(message: ChatCompletionMessage):
    if message.tool_calls and len(message.tool_calls) > 0:
        messages.append(
            {
                "role": "assistant",
                "content": message.content or "",
                "tool_calls": [
                    {
                        "id": tool_call.id,
                        "type": "function",
                        "function": {
                            "name": tool_call.function.name,
                            "arguments": tool_call.function.arguments,
                        },
                    }
                    for tool_call in message.tool_calls
                ],
            }
        )

        for tool_call in message.tool_calls:
            function_name = tool_call.function.name
            arguments = tool_call.function.arguments

            print(f"Calling function: '{function_name}' with arguments: {arguments}")

            try:
                arguments = json.loads(arguments)
            except json.JSONDecodeError:
                print(f"Error parsing arguments: {arguments}")
                arguments = {}

            function_result = FUNCTION_MAP[function_name](**arguments)
            print(f"Function result: {function_result}")

            messages.append(
                {
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "name": function_name,
                    "content": function_result,
                }
            )

        call_ai()

    else:
        messages.append({"role": "assistant", "content": message.content})
        print(f"Assistant: {message.content}")


def call_ai():
    response = client.chat.completions.create(
        model="gpt-5-nano-2025-08-07",
        messages=messages,
        tools=TOOLS,
    )

    process_ai_response(response.choices[0].message)


In [5]:
while True:
    message = input("Send a message to the LLM...")
    if message == "quit" or message == "q":
        break
    else:
        messages.append({"role": "user", "content": message})
        print(f"User: {message}")
        call_ai()


User: 안녕, 나는 송진욱이야
Assistant: 반갑습니다, 송진욱님! 오늘은 어떻게 도와드릴까요? 원하시는 것이 있으면 말씀해 주세요. 예를 들면 날씨 확인, 일정 관리, 번역, 정보 검색, 글쓰기 도움 등 어떤 것이든 도와드리겠습니다. 편하신 호칭이 있다면 알려 주세요.
User: 일본 날씨 알려줘
Assistant: 일본의 어느 도시의 날씨를 원하시는가요? 도시 이름을 하나 알려 주세요. 예: 도쿄, 오사카, 삿포로, 나고야, 교토, 후쿠오카 등. 여러 도시를 함께 알고 싶으시면 도시 이름을 쉼표로 구분해 주세요. 도시를 알려주시면 현재 기온과 오늘/주간 예보를 알려드리겠습니다.
User: 후쿠오카
Calling function: 'get_weather' with arguments: {"city":"후쿠오카"}
Function result: The weather in 후쿠오카 is sunny.
Assistant: 후쿠오카의 현재 날씨는 맑습니다.

원하시면 현재 기온이나 체감온도, 오늘/주간 예보 등 자세한 정보를 함께 알려드릴게요. 시간대별 예보나 비 올 확률도 확인해 드립니다. 다른 도시의 날씨도 필요하시면 말씀해 주세요.


In [6]:
messages

[{'role': 'user', 'content': '안녕, 나는 송진욱이야'},
 {'role': 'assistant',
  'content': '반갑습니다, 송진욱님! 오늘은 어떻게 도와드릴까요? 원하시는 것이 있으면 말씀해 주세요. 예를 들면 날씨 확인, 일정 관리, 번역, 정보 검색, 글쓰기 도움 등 어떤 것이든 도와드리겠습니다. 편하신 호칭이 있다면 알려 주세요.'},
 {'role': 'user', 'content': '일본 날씨 알려줘'},
 {'role': 'assistant',
  'content': '일본의 어느 도시의 날씨를 원하시는가요? 도시 이름을 하나 알려 주세요. 예: 도쿄, 오사카, 삿포로, 나고야, 교토, 후쿠오카 등. 여러 도시를 함께 알고 싶으시면 도시 이름을 쉼표로 구분해 주세요. 도시를 알려주시면 현재 기온과 오늘/주간 예보를 알려드리겠습니다.'},
 {'role': 'user', 'content': '후쿠오카'},
 {'role': 'assistant',
  'content': '',
  'tool_calls': [{'id': 'call_BvfpG9XcXbLTVy71ZiUTatyg',
    'type': 'function',
    'function': {'name': 'get_weather', 'arguments': '{"city":"후쿠오카"}'}}]},
 {'role': 'tool',
  'tool_call_id': 'call_BvfpG9XcXbLTVy71ZiUTatyg',
  'name': 'get_weather',
  'content': 'The weather in 후쿠오카 is sunny.'},
 {'role': 'assistant',
  'content': '후쿠오카의 현재 날씨는 맑습니다.\n\n원하시면 현재 기온이나 체감온도, 오늘/주간 예보 등 자세한 정보를 함께 알려드릴게요. 시간대별 예보나 비 올 확률도 확인해 드립니다. 다른 도시의 날씨도 필요하시면 말씀해 주세요.'}]